<span style="font-size:1.15em;font-weight:bold">
Your whole solution to the exercise should be implemented in this
file.

Be sure to read all comments in this file</span>

Your task is to first implement the functions `parse_json`,
`parse_timestamp`, `get_uuid`, `parse_bson`, `parse_xml`, and
`parse_volts` based on the given instructions.

Finally implement the main `solution` function.

Only the libraries mentioned in the Pipfile (in addition to the python
standard libraries) will be available in our grading setup



In [431]:
%pip install bson

Note: you may need to restart the kernel to use updated packages.


In [432]:
import ETCE.GroupA.Ex04.weather_data as wd  # Do not remove this import
from pprint import pprint

In [433]:
import json
import bson
import xml.etree.ElementTree as ET
from datetime import datetime
import pandas as pd
import uuid

Implement this function to parse the json weather data here,
and return a dictionary as follows:


    {'temp': ...,
     'clouds', ...,
     'wind_speed': ...,
     'wind_deg': ...,
     'wind_gust': ...,
     'rain': ...
    }


In [434]:

def parse_json(string_returned) -> dict:
    return json.loads(string_returned)

Implement this function, to parse the string timestamp 'yyyy-mm-dd hh:mm:ss.ms CEST' 
to a UNIX timestamp in UTC

In [435]:

def parse_timestamp(string_timestamp) -> int:
    # return int(datetime.timestamp(datetime.strptime(string_timestamp, '%Y-%m-%d %H:%M:%S.%f %Z')))
    return int(datetime.timestamp(datetime.strptime(string_timestamp, '%Y-%m-%d %H:%M:%S.%f')))

In [436]:
timestamp_str = '2024-05-06 12:30:00.000 CEST'
parse_timestamp(timestamp_str)

ValueError: unconverted data remains:  CEST

Implement this function to get a UUID using the uuid module

In [ ]:
def get_uuid() -> str:
    return str(uuid.uuid4())

Implement this function to parse the bson weather data here,
and return a dictionary as follows:


    {'temp': ...,
     'clouds', ...,
     'wind_speed': ...,
     'wind_deg': ...,
     'wind_gust': ...,
     'rain': ...
    }

In [ ]:
def parse_bson(string_returned) -> dict:
    return bson.decode(string_returned)

Implement this function to parse the xml weather data here,
and return a dictionary as follows:


    {'temp': ...,
     'clouds', ...,
     'wind_speed': ...,
     'wind_deg': ...,
     'wind_gust': ...,
     'rain': ...
    }

In [ ]:
def parse_xml(string_returned) -> dict:
    root = ET.fromstring(string_returned)
    d = {}
    for child in root:
        d[child.tag] = child.text
    return d

Implement this function to parse the xml voltage data from the
TMP36 sensor here, convert the voltage to temperature in °C and
return a dictionary as follows:    

{'temp': ....}

Hint: Search for a datasheet for a TMP36 sensor that shows the
relationship between a voltage reading from the sensor and the
temperature being sensed.

In [ ]:
with open('/media/storage/TU-Clausthal/ETCE/ETCE-Exercises/E04/ETCE/GroupA/Ex04/multi_format/ws4/7.xml', 'r') as xmlfile:
    volt = xmlfile.readline()

a = parse_xml(volt)
a

In [ ]:
# https://learn.adafruit.com/tmp36-temperature-sensor/overview
def parse_volts(string_returned)-> dict:
    
    volt = float(parse_xml(string_returned)['volts'])
    temp = (volt-500)/10
    return {'temp': temp}

In [ ]:
with open('/media/storage/TU-Clausthal/ETCE/ETCE-Exercises/E04/ETCE/GroupA/Ex04/multi_format/ws5/4.xml', 'r') as xmlfile:
    volt = xmlfile.readline()

a = parse_volts(volt)
a

This function should return the solution to E03.

This function should return a dictionary as follows:

    {0 : {"uuid" : "some-uuid",   Should be unique for every entry
          "timestamp": 00000000,  Some valid UNIX timestamp
           "temp":5.33,
           "clouds":100.0,
           "wind_speed":2.48,
           "wind_deg":31.0,
           "wind_gust":4.1,
           "rain":1.54
           } ,
     1 : {...},
     .
     .
     .
     n : {...}
     }

where n=total no of timestamps

Remember that you are combining data from various sources into one
dict for each timestamp.

When new data from multiple sources is available for the same
field in the data structure, compute the average. For example, all
sources give you temperature data, so you must return the average
temperature for each timestamp.

Also remember that you need to convert the voltage values for the
TMP36 sensor into temperature in °C before averaging with the
other temperature values.

Side note: Some cloudiness (%) data might go above 100 or below
0. You can simply ignore this mistake and proceed assuming a value
like 100.05 is correct.

In [ ]:
def solution()-> dict:
    # You can iteratively populate the solution_dict
    solution_dict:dict = {}
    i = 0

    while(True):
        try:
            ret, ts = wd.get_next()
            # This function simulates an API query to get the weather
            # data.

            # `ret` is a tuple containing weather data from 5 sources,
            # (Source 1, Source 2, Source 3, Source 4, Source 5) at
            # timestamp `ts`

            # ts is the string timestamp when the data was recorded.

            # The structure of `ret` is self-documented, and contains
            # all the info you need to parse the data.
            
            solution_dict[i]= {}
            solution_dict[i]["uuid"] = get_uuid()
            solution_dict[i]["timestamp"] = parse_timestamp(ts)
            
            b = parse_json(ret[i]["data"])
            solution_dict[i].update(b)
            
            # You can read the data returned by the API as follows
            # pprint(ret[0]["description"]) # Reading the description of the first weather station
            pprint(ret[0]["data"]) # Reading the description of the first weather station
            print(type(ret[0]["data"])) # Reading the datatype of the first weather station data
            print(ts)
            if( i==2):
                break
            i+=1
            break

        except Exception as E:
            print(E)
            break

    return solution_dict
solution()

### Evaluate your solution

In [ ]:
from driver import evaluate
wd.i = 0
evaluate(solution, parse_json, parse_bson, parse_xml, parse_timestamp, get_uuid, parse_volts)